In [1]:
import logging
import warnings
logging.getLogger('fbprophet').setLevel(logging.ERROR)
logging.getLogger('statsmodels').setLevel(logging.WARNING)
logging.getLogger('arch').setLevel(logging.WARNING)
logging.getLogger('croston').setLevel(logging.WARNING)
logging.getLogger('pandas').setLevel(logging.WARNING)
logging.getLogger('pmdarima').setLevel(logging.WARNING)
logging.getLogger('causalimpact').setLevel(logging.WARNING)
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [2]:
import forecast_engine.modelling.utils.nodes as utils
import forecast_engine.modelling.forecast_engine.forecast_loop as forecast_loop
import forecast_engine.modelling.forecast_engine.forecast_engine_pipeline as forecast_engine_pipeline





import forecast_engine.modelling.models.ml_models.naive_models as naive_models
import forecast_engine.modelling.models.ml_models.holt_winters as holt_winters

import forecast_engine.modelling.models.ml_models.garch as garch
import forecast_engine.modelling.models.ml_models.rw_drift as rw_drift
import forecast_engine.modelling.models.ml_models.arima as arima
import forecast_engine.modelling.models.ml_models.croston as croston

import forecast_engine.modelling.models.ml_models.causal_impact as causal_impact
import forecast_engine.modelling.models.ml_models.prophet as prophet

import forecast_engine.modelling.models.ml_models.light_gbm as light_gbm
import forecast_engine.modelling.models.ml_models.xgboost as xgboost




import forecast_engine.modelling.models.ml_models.linear_regression_mv as linear_regression_mv
import forecast_engine.modelling.models.ml_models.arima_mv as arima_mv
import forecast_engine.modelling.models.ml_models.causal_impact_mv as causal_impact_mv
import forecast_engine.modelling.models.ml_models.xgboost_mv as xgboost_mv
import forecast_engine.modelling.models.ml_models.light_gbm_mv as light_gbm_mv
import forecast_engine.modelling.models.ml_models.prophet_mv as prophet_mv
import forecast_engine.modelling.models.ml_models.olsregression_mv as olsregression_mv
import forecast_engine.modelling.models.ml_models.random_forest_mv as random_forest_mv




import pandas as pd
import numpy as np
import datetime


PREPROCESS

In [3]:
import forecast_engine.preprocess.nodes as preprocess

SALES ORDER

In [4]:
path = r"/Users/anand_k/Downloads/UPL/Demand forecasting datasets [DO NOT EDIT]/Sales order"
sales_data, df = preprocess.read_prep_sales_order(path)
dates, act_dates = preprocess.prep_dates_list(df)

In [5]:
sub_season = [str(x)+'_Season' for x in dates]
df[sub_season] = df.apply( lambda x : preprocess.decomposition(x,dates,df.columns), axis = 1, result_type="expand")

In [6]:
df['intermittence'] = df.apply(lambda x : preprocess.intermittence(x), axis =1)
df['cov'] = df[dates].apply(lambda x : preprocess.cov(x), axis =1)
df['seq - flag'] = df.apply(lambda x : preprocess.seq_flag(x,dates,df.columns) , axis = 1)

In [7]:
df.drop(['intermittence','cov'],axis =1,inplace=True)

WEATHER DATA

In [8]:
path = r"/Users/anand_k/Downloads/UPL/Demand forecasting datasets [DO NOT EDIT]/Weather Data/Weather Data.xlsx"
wthr_data = preprocess.read_prep_weather(path)

In [9]:
wthr_data

,Country,2019 - 01_Avg Temp (C),2019 - 02_Avg Temp (C),2019 - 03_Avg Temp (C),2019 - 04_Avg Temp (C),2019 - 05_Avg Temp (C),2019 - 06_Avg Temp (C),2019 - 07_Avg Temp (C),2019 - 08_Avg Temp (C),2019 - 09_Avg Temp (C),...,2021 - 10_Relative Humidity (%),2021 - 11_Relative Humidity (%),2021 - 12_Relative Humidity (%),2022 - 01_Relative Humidity (%),2022 - 02_Relative Humidity (%),2022 - 03_Relative Humidity (%),2022 - 04_Relative Humidity (%),2022 - 05_Relative Humidity (%),2022 - 06_Relative Humidity (%),2022 - 07_Relative Humidity (%)
0,Mexico,15.100685,17.501682,18.93809,20.299129,22.072156,21.325376,20.17738,20.719087,19.970685,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732


MACROECONOMIC DATA

In [10]:
path = r"/Users/anand_k/Downloads/UPL/Demand forecasting datasets [DO NOT EDIT]/Macro-economics"
macro = preprocess.read_macro_files(path, country = ['Mexico'])

In [11]:
macro

,2019 - 01_Inflation Data,2019 - 02_Inflation Data,2019 - 03_Inflation Data,2019 - 04_Inflation Data,2019 - 05_Inflation Data,2019 - 06_Inflation Data,2019 - 07_Inflation Data,2019 - 08_Inflation Data,2019 - 09_Inflation Data,2019 - 10_Inflation Data,...,2022 - 04_Lending Rate,2022 - 05_Lending Rate,2022 - 06_Lending Rate,2022 - 07_Lending Rate,2022 - 08_Lending Rate,2022 - 09_Lending Rate,2022 - 10_Lending Rate,2022 - 11_Lending Rate,2022 - 12_Lending Rate,Country
0,0.0437,0.0394,0.04,0.0441,0.0428,0.0395,0.0378,0.0316,0.03,0.0302,...,0.065,0.07,0.0775,0.0775,0.085,0.0925,0.0925,0.1,0.105,Mexico


MERGE DATA

In [12]:
df['Country'] = 'Mexico'
data = pd.merge(df, macro, on = 'Country', how = 'left')
data = pd.merge(data, wthr_data, on = 'Country', how = 'left')
data.drop('Country',axis=1,inplace=True)
data.rename(columns={'Material':'index'},inplace=True)
data.columns = [str(i) for i in list(data.columns) ]
    

In [13]:
data

,index,2019 - 03,2019 - 04,2019 - 05,2019 - 06,2019 - 07,2019 - 08,2019 - 09,2019 - 10,2019 - 11,...,2021 - 10_Relative Humidity (%),2021 - 11_Relative Humidity (%),2021 - 12_Relative Humidity (%),2022 - 01_Relative Humidity (%),2022 - 02_Relative Humidity (%),2022 - 03_Relative Humidity (%),2022 - 04_Relative Humidity (%),2022 - 05_Relative Humidity (%),2022 - 06_Relative Humidity (%),2022 - 07_Relative Humidity (%)
0,U00000000000180123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
1,U00000000000208289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
2,U00000000000368011,NaN,768.0,1044.0,1824.0,1128.0,1428.0,1824.0,1228.0,564.0,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
3,U00000000000554501,NaN,10000.0,19250.0,5000.0,NaN,NaN,NaN,3500.0,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
4,U00000000000563501,NaN,600.0,4300.0,2350.0,NaN,NaN,NaN,4450.0,2000.0,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,U00000000189060238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
1217,U00000000189060244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
1218,U00000000189060261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732
1219,U00000000189060268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,74.688081,67.689772,61.966188,63.661877,58.495351,49.211327,53.776839,52.436179,68.25951,72.98732


Forecast Engine

In [72]:
#Flags
debug_flag = 0
debug_models = []
outlier_flag = 1
select_variable_flag = 0
all_at_once_flag = 1
months_to_forecast = 3
max_validation = 12
validation_window = 4
single_series = 1
multivariate_flag = 1

univariate_models = {'hw_ets_r_MNN':holt_winters.hw_ets_r_MNN, 'hw_ets_ATA':holt_winters.hw_ets_ATA, 'hw_ets_AFA':holt_winters.hw_ets_AFA, 
                    'hw_ets_ATN':holt_winters.hw_ets_ATN, 'hw_ets_NFA':holt_winters.hw_ets_NFA, 'hw_ets_NFN':holt_winters.hw_ets_NFN, 'hw_ets_AFN':holt_winters.hw_ets_AFN,
                    'hw_ets_r_MAA':holt_winters.hw_ets_r_MAA,'hw_ets_r_MAN':holt_winters.hw_ets_r_MAN, 'hw_ets_r_MNA':holt_winters.hw_ets_r_MNA,
                    
                    'mean_model':naive_models.mean_model, 'median_model':naive_models.median_model, 'naive':naive_models.naive, 'naive_ets':naive_models.naive_ets,
                    'snaive':naive_models.snaive, 'snaive_twoseasons':naive_models.snaive_twoseasons,
                    
                    'rw_drift':rw_drift.rw_drift, 'croston':croston.croston, 'arima':arima.arima, 'garch':garch.garch, 'linearregression_mv':linear_regression_mv.linearregression_mv,

                    'xgboost_mv':xgboost_mv.xgboost_mv,'prophet':prophet.prophet,'causalimpact':causal_impact.causalimpact,'light_gbm':light_gbm.lgbm,
                    'random_forest_mv' : random_forest_mv.random_forest_mv}


multivariate_models = {'linearregression_mv':linear_regression_mv.linearregression_mv, 'arima_mv':arima_mv.arima_mv, 
                        'xgboost_mv':xgboost_mv.xgboost_mv, 'prophet_mv':prophet_mv.prophet_mv, 'olsregression_mv':olsregression_mv.olsregression_mv, 
                        'random_forest_mv' : random_forest_mv.random_forest_mv}
                        #TODO 'light_gbm_mv':light_gbm_mv.lgbm_mv, 'causalimpact_mv':causal_impact_mv.causalimpact_mv
                    

list_of_models = univariate_models.copy()
if multivariate_flag == 1:  list_of_models.update(multivariate_models)


In [73]:
df_output = data[data['index']=='U00000000006030126'].apply(lambda x: forecast_engine_pipeline.forecast_engine(x, months_to_forecast,debug_flag,debug_models,list_of_models, single_series,  all_at_once_flag, validation_window, max_validation, outlier_flag, select_variable_flag),axis = 1)    

U00000000006030126
       Header   Value  Avg Temp (C) Inflation Data Lending Rate  Max Temp (C)  \
38  2022 - 05   216.0     22.117168         0.0765         0.07     32.208969   
39  2022 - 06   432.0     20.450507         0.0799       0.0775     30.314979   
40  2022 - 07   228.0     20.463593         0.0815       0.0775     29.184962   
41  2022 - 08   468.0     20.463593          0.087        0.085     29.184962   
42  2022 - 09  1153.0     20.463593          0.087       0.0925     29.184962   
43  2022 - 10   240.0     20.463593         0.0841       0.0925     29.184962   
44  2022 - 11  1320.0     20.463593          0.078          0.1     29.184962   
45  2022 - 12  2328.0     20.463593         0.0782        0.105     29.184962   
46  2023 - 01     NaN           NaN            NaN          NaN           NaN   
47  2023 - 02     NaN           NaN            NaN          NaN           NaN   
48  2023 - 03     NaN           NaN            NaN          NaN           NaN   
49  2023 

/Users/anand_k/Downloads/UPL/Forecast Engine/forecast_engine/modelling/models/ml_models/holt_winters.py:59: FutureWarning: the 'damped'' keyword is deprecated, use 'damped_trend' instead.
  fit = ets(cv_train.values,trend=self.trend,damped=self.damped,seasonal=self.seasonality,seasonal_periods=seasonal_periods).fit(optimized=True, start_params=None)
/Users/anand_k/Downloads/UPL/Forecast Engine/forecast_engine/modelling/models/ml_models/holt_winters.py:59: FutureWarning: the 'damped'' keyword is deprecated, use 'damped_trend' instead.
  fit = ets(cv_train.values,trend=self.trend,damped=self.damped,seasonal=self.seasonality,seasonal_periods=seasonal_periods).fit(optimized=True, start_params=None)
/Users/anand_k/Downloads/UPL/Forecast Engine/forecast_engine/modelling/models/ml_models/holt_winters.py:59: FutureWarning: the 'damped'' keyword is deprecated, use 'damped_trend' instead.
  fit = ets(cv_train.values,trend=self.trend,damped=self.damped,seasonal=self.seasonality,seasonal_periods=s

Optimization not converged


/Users/anand_k/Downloads/UPL/Forecast Engine/forecast_engine/modelling/models/ml_models/linear_regression_mv.py:104: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  feature_scores = {'linearregression_mv':(pd.Series(index=regressors.columns).sort_values(ascending=False))}


In [69]:
df_output = df_output.ravel()
output_temp = df_output[0]['output']
df_classifier_train_set = df_output[0]['classifier_train_set']
impact_temp = df_output[0]['var_coeff']
df_univariate_temp = df_output[0]['df_univariate']
mp = df_output[0]['df_multivariate']
df_ensemble_temp = df_output[0]['df_ensemble']
df_fitted_values_list = df_output[0]['fitted_values_list']
model_list = df_output[0]['dataf']

In [70]:
output_temp

,index,DATE,Forecast,Method,Forecast_accuracy,Actual,timing,pre_timing,mid_timing,post_timing,total
0,U00000000006030126,2022 - 10,936.494159,linearregression_mv,26.811583,240.0,0.454092,0.016364,0.458846,0.004314,0.479524
1,U00000000006030126,2022 - 11,1069.426884,linearregression_mv,26.811583,1320.0,0.454092,0.016364,0.458846,0.004314,0.479524
2,U00000000006030126,2022 - 12,971.232684,linearregression_mv,26.811583,2328.0,0.454092,0.016364,0.458846,0.004314,0.479524
3,U00000000006030126,2023 - 01,922.969450,linearregression_mv,NaN,NaN,0.454092,0.016364,0.458846,0.004314,0.479524
4,U00000000006030126,2023 - 02,645.753686,linearregression_mv,NaN,NaN,0.454092,0.016364,0.458846,0.004314,0.479524
5,U00000000006030126,2023 - 03,474.149437,linearregression_mv,NaN,NaN,0.454092,0.016364,0.458846,0.004314,0.479524


In [71]:
model_list

,models,smape
0,linearregression_mv,26.811583
1,mean_model,39.138449
2,median_model,39.869652


In [61]:
df_fitted_values_list

[OrderedDict([('mean_model',
               {'forecast':         date_     forecast      method
                0   2019 - 03    36.000000  mean_model
                1   2019 - 04   185.000000  mean_model
                2   2019 - 05   535.333333  mean_model
                3   2019 - 06   521.500000  mean_model
                4   2019 - 07   575.600000  mean_model
                5   2019 - 08   709.666667  mean_model
                6   2019 - 09   714.571429  mean_model
                7   2019 - 10   839.750000  mean_model
                8   2019 - 11  1053.111111  mean_model
                9   2019 - 12  1107.400000  mean_model
                10  2020 - 01  1318.713733  mean_model
                11  2020 - 02  1355.820922  mean_model
                12  2020 - 03  1448.142389  mean_model
                13  2020 - 04  1361.846504  mean_model
                14  2020 - 05  1301.456737  mean_model
                15  2020 - 06  1275.740691  mean_model
                16  2020

DEGUG MODELS

In [36]:
ti = data[data['index']=='U00000000006030126'].squeeze(axis=0)
ti_series = ti[ti.index.str.contains(' - ') & ~(ti.index.str.contains('_'))]

series_flag = ti_series[(ti_series.size-1)]
ti_key = ti[~(ti.index.str.contains(' ')) & ~(ti.index.str.contains(' - '))]


ti_vars =ti[(ti.index.str.contains(' ')) & (ti.index.str.contains('_'))]
ti_vars = ti[(ti.index.str.contains(' ')) & (ti.index.str.contains('_'))]
ti = pd.concat([ti_key,ti_series])


ti_vars = pd.DataFrame(ti_vars).reset_index()
ti_vars['month_id'] = pd.Series()
ti_vars['driver_desc'] = pd.Series()

if ti_vars.shape[0]>0:
    ti_vars['month_id'] = ti_vars['index'].str.split("_", expand=True)[0]
    ti_vars['driver_desc'] = ti_vars['index'].str.split("_", expand=True)[1]

ti_vars.columns = ['index','value','month_id','driver_desc']
drivers_local = ti_vars.drop(['index'],axis=1)
drivers_local = drivers_local[['month_id','driver_desc','value']]

#   print('drivers_local is', drivers_local)

drivers_local = pd.DataFrame(drivers_local.pivot(index='month_id',columns='driver_desc'))
drivers_local.columns = drivers_local.columns.droplevel()
drivers_local = drivers_local.reset_index()
drivers_local['month_id'] = drivers_local['month_id'].astype(str)
drivers_local = drivers_local.dropna(axis=1, how = 'all')
variable_list = pd.Series(drivers_local.columns[1:])


ti=pd.DataFrame([ti]).T
ti['Value']=ti.index
ti=ti.reset_index(drop=True)
ti=ti[0:(ti.shape[0]-1)]
ti.columns=["Value","Header"]
ti = ti[ti['Header'].str.contains(' - ')]
ti.index=ti['Header']
ti = ti.iloc[:,0]
ti = pd.DataFrame(ti)
series_index = ti.index

if ti_vars.shape[0]>0:
    ti = pd.merge(ti,drivers_local,left_on=series_index,right_on=drivers_local['month_id'],how="outer")
    ti['Header'] = ti['key_0']
    ti = ti.drop(['month_id','key_0'],axis=1)
    cols = list(ti.columns)
    cols = [cols[-1]] + cols[:-1]
    ti = ti[cols]
    ti.sort_values('Header', inplace=True)
else: 
    ti = ti.reset_index()

In [37]:
ti = ti[2:]

ti1 = utils.actual_imputation(ti.copy())
ti=ti1.copy()

ti2 = utils.driver_imputation(ti1.copy())
ti=ti2.copy()

In [38]:
daddy_df = ti.copy()

In [39]:
import datetime
ts1 = datetime.datetime.now().timestamp()
import forecast_engine.modelling.anomaly_detection.pipeline as anomaly_pipeline
import forecast_engine.modelling.forecast_engine.variable_selection as variable_selection

ti =ti[2:]
row_counter = ti.shape[0] +1
fitted_values_list=list()
################################################################# FOR ANALYSIS MODULE IN ANY ENV ######################################################################
df_multivariate = pd.DataFrame()
df_univariate = pd.DataFrame()
df_ensemble = pd.DataFrame()
################################################################# FOR ANALYSIS MODULE IN ANY ENV ######################################################################

oos_length = 1
if all_at_once_flag == 1:
    oos_length = months_to_forecast
    months_to_forecast = 1
cols_proper = list(ti.columns)
for i in range(0,months_to_forecast):
    if debug_flag==1: print('Run Number '+str(i))
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    
    ti = ti.append(pd.Series(np.nan),ignore_index=True)    

ti = ti[cols_proper]
ti['Header'] = pd.to_datetime(ti['Header'],format="%Y - %m")
ti['Header'] = pd.date_range(start=ti['Header'][0],periods=(ti.shape[0]),freq='MS').strftime('%Y - %m')


import collections
import random
import itertools
from collections import OrderedDict
from copy import deepcopy

n_splits = int(np.floor(np.minimum(max_validation,(len(ti['Value'][:(row_counter-1)].dropna())-1))/validation_window))
index = ti['Value'].first_valid_index()

if n_splits==0: series_flag = 5
if debug_flag==1: print(n_splits)
if debug_flag==1: print(series_flag)


ti = ti.dropna(axis=1, how = 'all')
for i in ti.iloc[:,2:].columns:
    col_length = pd.Series(ti[i].unique()).dropna().shape[0]
    if col_length == 1:
        ti = ti.drop([i],axis=1)

ti1 = ti.copy()  

anomalous_months = []

if outlier_flag == 1:
    if series_flag==1:
            tad = anomaly_pipeline.twitteranomalydetection(ti1,plot=False)
            anomalous_months = tad['anomalies'].index
            ti1['Value'][ti1['Header'].isin(anomalous_months)] = tad['anomalies']['expected_value'].values
    

######### VARIABLE SELECTION #########
if select_variable_flag == 1:
    ti_select= variable_selection.variable_selection(ti1.copy(),row_counter)
    ti1=ti_select.copy()  
######### VARIABLE SELECTION #########


ti_mv = ti.copy()
ti_mv.index=ti_mv['Header']
ti_mv = ti_mv.drop(['Header'],axis=1)
ti.index = ti['Header']
ti = ti['Value'].copy()
multivariate_models = ['arima_mv','xgboost_mv','lightgbm_mv','causalimpact_mv','prophet_mv','linearregression_mv','olsregression_mv','random_forest_mv']
model_list=list()
val_forecast_list=collections.OrderedDict()
oos_forecast_list=collections.OrderedDict()
fitted_values_list=collections.OrderedDict()
var_coeff=collections.OrderedDict()

val_forecast_list = deepcopy(val_forecast_list)
oos_forecast_list = deepcopy(oos_forecast_list)
fitted_values_list = deepcopy(fitted_values_list)
feature_scores= collections.OrderedDict()
h = oos_length
import forecast_engine.modelling.utils.nodes as utils
import forecast_engine.modelling.error_metrics.nodes as error_metrics

<Figure size 640x480 with 0 Axes>

In [40]:
import forecast_engine.modelling.forecast_engine.individual_model_run as individual_model_run

for model_name in list_of_models:
    if debug_flag==1: print(model_name)
    ts1 = datetime.datetime.now().timestamp()
    model_name,val_forecast_list,oos_forecast_list,var_coeff,fitted_values_list = individual_model_run.individual_model_run_function(model_name,multivariate_models,list_of_models,ti_mv,ti,oos_length,row_counter,debug_models,n_splits,validation_window,variable_list,model_list,val_forecast_list,oos_forecast_list,fitted_values_list,var_coeff)
    ts2 = datetime.datetime.now().timestamp()
    if debug_flag==1: print(ts2 - ts1)
        

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.36887D+00    |proj g|=  4.20723D+01

At iterate    5    f=  4.96570D-01    |proj g|=  3.63280D+00

At iterate   10    f=  3.70675D-01    |proj g|=  3.72051D-01

At iterate   15    f=  1.41070D-01    |proj g|=  5.94338D-01

At iterate   20    f=  6.76752D-02    |proj g|=  1.17841D-01

At iterate   25    f=  4.63520D-02    |proj g|=  1.13154D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     27     45      1     0     0   

 This problem is unconstrained.
 This problem is unconstrained.


At iterate   15    f=  2.58130D-02    |proj g|=  1.33333D-01

At iterate   20    f=  1.92033D-02    |proj g|=  2.85617D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     22     39      1     0     0   3.832D-06   1.920D-02
  F =   1.9203069547466436E-002

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.69447D+00    |proj g|=  1.07367D+01

At iterate    5    f=  1.38601D-01    |proj g|=  2.13475D-01

At itera

 This problem is unconstrained.


In [41]:
ti = daddy_df
ti1 = ti

In [42]:
if outlier_flag == 1:
    if series_flag==1:
        ti1 = ti1[~ti1.index.isin(anomalous_months)]
    

models_smape = OrderedDict()

validation_len = min([y for y in [len(val_forecast_list[x]['forecast']) for x in list(val_forecast_list.keys())]])
for i in val_forecast_list.keys():
    val_forecast_list[i]['forecast'] = val_forecast_list[i]['forecast'][-validation_len:].reset_index(drop=True)

smape_weights = pd.Series([1]*validation_len,name='weights')
for i in model_list:
    y_pred = val_forecast_list[i]['forecast'].copy()
    y_pred = y_pred[~y_pred['date_'].isin(anomalous_months)]
    y_act = ti1[ti1['Header'].isin(list(y_pred['date_']))]
    
    if debug_flag==1:
        print(i)
        print(y_pred)    


    models_smape[i] = error_metrics.wsmape(y_act['Value'],y_pred['forecast'], smape_weights[0:len(y_act)])

In [43]:
dataf = pd.DataFrame.from_dict(models_smape, orient='index').reset_index()
dataf.columns = ['models','smape']
dataf = dataf.sort_values(by = 'smape').reset_index(drop=True)
print(dataf)

                models      smape
0               snaive  23.649433
1  linearregression_mv  29.472237
2                garch  32.711915
3                arima  34.385042
4    snaive_twoseasons  36.642999
5           mean_model  39.632843
6            naive_ets  39.632998
7         median_model  39.754574
8                naive  46.969824
9              croston  46.969828


In [26]:
ti = daddy_df

In [27]:
ti_backup=ti.copy()
ti = ti[index:(row_counter-1)]

models = dataf[~dataf['models'].isin(['xgboost_mv','xgboost'])].iloc[0:5,0].reset_index(drop=True).tolist()
classifier_train_set = pd.DataFrame(list(list_of_models.keys())).T
classifier_train_set.columns = classifier_train_set.iloc[0]
for i in classifier_train_set.columns:
    classifier_train_set[i] = int(pd.Series(i).isin(models))

list_of_models = dict((k, list_of_models[k]) for k in models)
best_model = dataf.iloc[0:1,0].reset_index(drop=True)[0]
best_model_smape = dataf.iloc[0:1,1].reset_index(drop=True)[0]

oos_forecast = oos_forecast_list[best_model]['forecast'].copy()
oos_forecast['Actual'] = np.nan
print('oos_forecast before:\n',oos_forecast)
    
val_length = val_forecast_list[best_model]['forecast'].shape[0]
x = ti[(ti.shape[0]-val_length):(ti.shape[0])]['Value']
val_forecast = val_forecast_list[best_model]['forecast'].copy()
val_forecast['Forecast_accuracy'] = best_model_smape
val_forecast.loc[:,'Actual'] = x.values
oos_forecast.rename(columns = {'method':'type'}, inplace=True)
val_forecast.rename(columns = {'method':'type'}, inplace=True)

oos_forecast before:
        date_     forecast      method  Actual
0  2023 - 01  1836.842004  hw_ets_AFA     NaN
1  2023 - 02  1772.801066  hw_ets_AFA     NaN
2  2023 - 03  1504.760277  hw_ets_AFA     NaN


In [28]:
import forecast_engine.modelling.forecast_engine.analysis_dataframe as analysis_dataframe

In [29]:
best_ensemble=pd.DataFrame();ensemble_1=pd.DataFrame();ensemble_2=pd.DataFrame();ensemble_3=pd.DataFrame();ensemble_4=pd.DataFrame()
import forecast_engine.modelling.models.ensembles.ensemble_engine as ensemble_engine 
if series_flag==1:
    ensemble_1=ensemble_engine.simple_mean_ensemble(oos_forecast, val_forecast, val_forecast_list, oos_forecast_list, list_of_models, smape_weights, anomalous_months)
    ensemble_2=ensemble_engine.weighted_mean_ensemble(oos_forecast,val_forecast,val_forecast_list,oos_forecast_list,list_of_models,dataf,smape_weights, anomalous_months)
    ensemble_3=ensemble_engine.linear_regressor_ensemble(oos_forecast,val_forecast,val_forecast_list,oos_forecast_list,list_of_models,smape_weights, anomalous_months)
    ensemble_4=ensemble_engine.bagging_regressor_ensemble(oos_forecast,val_forecast,val_forecast_list,oos_forecast_list,list_of_models,smape_weights, anomalous_months)
    best_ensemble= pd.concat([val_forecast.iloc[0].T,ensemble_1.iloc[0].T,ensemble_2.iloc[0].T,ensemble_3.iloc[0].T],axis=1).T [['Forecast_accuracy','type']].reset_index(drop=True)
    ensemble_type=pd.Series(['val_forecast','ensemble_1','ensemble_2','ensemble_3'],name='ensemble_type')
    
    
    
    best_ensemble=pd.concat([best_ensemble,ensemble_type],axis=1)
    best_ensemble_name=best_ensemble.iloc[(np.where(best_ensemble.iloc[:,0]==(best_ensemble.iloc[:,0]).min()))[0].min(),2]  
    if best_ensemble_name!='val_forecast':
      val_forecast = eval(best_ensemble_name).iloc[0:eval(best_ensemble_name).shape[0]-oos_length,:]
      print('oos_forecast before:\n',oos_forecast)
      oos_forecast = pd.DataFrame(eval(best_ensemble_name).iloc[eval(best_ensemble_name).shape[0]-oos_length:,:])
      print('oos_forecast after:\n',oos_forecast)
      multivariate_models = [x for x in model_list if '_mv' in x]
      impact_models=list(set(list(list_of_models.keys())) & (set(multivariate_models)))
      if len(impact_models)>0:
        best_model_index=np.min([np.where(dataf['models']==x) for x in impact_models])
        best_mv_model=dataf['models'].iloc[best_model_index]
        var_coeff=var_coeff[best_mv_model]['forecast']
      else:
        var_coeff=var_coeff[models[0]]['forecast']
        var_coeff['Model']=', '.join(list_of_models)
    else: 
      var_coeff=var_coeff[best_model]['forecast']
  
else: 
    var_coeff=var_coeff[best_model]['forecast'] 
        


In [30]:
df_univariate, df_multivariate, df_ensemble = analysis_dataframe.get_analysis_dfs(dataf,oos_forecast_list,fitted_values_list,best_ensemble,val_forecast,ensemble_1,ensemble_2,ensemble_3,ensemble_4)
val_length = 3
if val_forecast.shape[0]>val_length:
    val_forecast = val_forecast[(val_forecast.shape[0]-val_length):val_forecast.shape[0]]
else:
    index=[ti_backup['Header'].values[row_counter-1-val_length+i] for i in range(0,val_length-val_forecast.shape[0])]
    for i in range(0,len(index)):
        extra=pd.DataFrame.from_dict({'date_':[index[i]],'forecast':[0],'type':['no_model'],'Forecast_accuracy':[0],'Actual':[0]})
        val_forecast=pd.concat([extra,val_forecast],axis=0)

val_forecast = val_forecast[(val_forecast.shape[0]-val_length):val_forecast.shape[0]]
all_forecasts = pd.concat([val_forecast,oos_forecast], axis=0)
all_forecasts = all_forecasts.reset_index(drop=True)

In [31]:
ti_key = pd.DataFrame([ti_key])
key_classifier_train_set = pd.concat([ti_key]*classifier_train_set.shape[0],axis=0, ignore_index=True)
classifier_train_set = pd.concat([key_classifier_train_set,classifier_train_set],axis=1)

################################################################# FOR ANALYSIS MODULE IN NON-PROD ENV ######################################################################
key_df_univariate = pd.concat([ti_key]*df_univariate.shape[0],axis=0, ignore_index=True)
df_univariate = pd.concat([key_df_univariate,df_univariate],axis=1)
key_df_multivariate = pd.concat([ti_key]*df_multivariate.shape[0],axis=0, ignore_index=True)
df_multivariate = pd.concat([key_df_multivariate,df_multivariate],axis=1)
key_df_ensemble = pd.concat([ti_key]*df_ensemble.shape[0],axis=0, ignore_index=True)
df_ensemble = pd.concat([key_df_ensemble,df_ensemble],axis=1)

In [35]:
try:
    key_var = pd.concat([ti_key]*var_coeff.shape[0],axis=0, ignore_index=True)
    var_coeff = pd.concat([key_var,var_coeff],axis=1)
except: pass
  
# ti_key = pd.concat([ti_key]*forecast_out.shape[0], ignore_index=True)
# output = pd.concat([ti_key,forecast_out],axis=1)

try:
    var_coeff['key'] = var_coeff['d1_id'].astype(str)+var_coeff['d2_id'].astype(str)+var_coeff['d3_id'].astype(str)+var_coeff['d4_id'].astype(str)+var_coeff['d5_id'].astype(str)+var_coeff['d6_id'].astype(str)+var_coeff['kpi_id'].astype(str)
except: pass
  
ti_imputed = ti[ti['Header'].isin(var_coeff['Date'].unique().tolist())]
ti_imputed = pd.melt(ti_imputed, id_vars = 'Header')
ti_imputed.columns = ['Date','Driver','Imputed_Value']
try:
    var_coeff = pd.merge(var_coeff,ti_imputed,on = ['Date','Driver'])
except:
    var_coeff = pd.DataFrame(columns=['index','d1_id','d2_id','d3_id','d4_id','d5_id','d6_id','kpi_id','currency_code','randNumCol','Model','Date','Driver','Impact','key','Imputed_Value'])


In [39]:
ti_imputed = ti[ti['Header'].isin(var_coeff['Date'].unique().tolist())]
ti_imputed = pd.melt(ti_imputed, id_vars = 'Header')
ti_imputed.columns = ['Date','Driver','Imputed_Value']

In [40]:
ti_imputed

,Date,Driver,Imputed_Value


DEBUG THE ENGINE

In [18]:
# Function test
ti = data[data['index']=='U00000000006030126'].squeeze(axis=0)
from pandas import MultiIndex
import datetime 
single_series = 0
ts1 = datetime.datetime.now().timestamp()
if (single_series == 0):
  ti = pd.Series(ti)
series_key = str(ti['index'])
print(series_key)
ti1, ti_key, variable_list, series_flag = utils.get_ti_in_shape(ti.copy())
ti = ti1.copy()
ts2 = datetime.datetime.now().timestamp()
ti1 = utils.driver_imputation(ti.copy())
ti=ti1.copy()
ti = ti[2:]
  
forecast_loop.forecast_loop_function(ti, series_flag, months_to_forecast, debug_flag, debug_models, list_of_models, variable_list, single_series, all_at_once_flag, validation_window, max_validation, outlier_flag, select_variable_flag)

U00000000006030126
       Header   Value  Avg Temp (C) Inflation Data Lending Rate  Max Temp (C)  \
38  2022 - 05   216.0     22.117168         0.0765         0.07     32.208969   
39  2022 - 06   432.0     20.450507         0.0799       0.0775     30.314979   
40  2022 - 07   228.0     20.463593         0.0815       0.0775     29.184962   
41  2022 - 08   468.0     20.463593          0.087        0.085     29.184962   
42  2022 - 09  1153.0     20.463593          0.087       0.0925     29.184962   
43  2022 - 10   240.0     20.463593         0.0841       0.0925     29.184962   
44  2022 - 11  1320.0     20.463593          0.078          0.1     29.184962   
45  2022 - 12  2328.0     20.463593         0.0782        0.105     29.184962   
46  2023 - 01     NaN           NaN            NaN          NaN           NaN   
47  2023 - 02     NaN           NaN            NaN          NaN           NaN   
48  2023 - 03     NaN           NaN            NaN          NaN           NaN   
49  2023 

/Users/anand_k/Downloads/UPL/Forecast Engine/venv/lib/python3.9/site-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


Optimization not converged
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.50001D+00    |proj g|=  1.34026D+01

At iterate    5    f=  1.43976D-01    |proj g|=  1.24640D+00

At iterate   10    f=  9.27024D-02    |proj g|=  1.38881D+00

At iterate   15    f=  2.23839D-02    |proj g|=  2.11599D-01

At iterate   20    f=  4.93587D-03    |proj g|=  4.95169D-02


 This problem is unconstrained.



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     24     29      1     0     0   3.651D-06   4.843D-03
  F =   4.8425208183474847E-003

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  9.55551D-01    |proj g|=  8.71739D+00

At iterate    5    f=  7.51862D-02    |proj g|=  4.81391D-01

At iterate   10    f=  3.89027D-02    |proj g|=  1.05487D+00

At iterate   15    f= -2.41006D-03    |proj g|=  1.10583D-01


...


At iterate   20    f= -3.34316D-03    |proj g|=  1.95525D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     21     24      1     0     0   7.439D-06  -3.343D-03
  F =  -3.3431634658283335E-003

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.48677D-01    |proj g|=  7.26294D+00

At iterate    5    f=  2.69010D-02    |proj g|=  2.44154D-01

At iterate   10    f= -4.04674D-02    |proj g|=  6.27288D-02

At iter

 This problem is unconstrained.


Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.51355D-01    |proj g|=  8.11998D+00

At iterate    5    f=  1.64728D-02    |proj g|=  2.04319D-01

At iterate   10    f= -2.78581D-02    |proj g|=  1.44435D+00

At iterate   15    f= -5.24658D-02    |proj g|=  1.48840D-01

At iterate   20    f= -5.82869D-02    |proj g|=  1.54551D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4     22     28      1     0     0   1.642D-05  -5.829D-02
  F =  -5.8287011460081618E-002

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH     

ValueError: operands could not be broadcast together with shapes (4,) (12,) 

<Figure size 640x480 with 0 Axes>